In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
from tensorflow import keras as ks

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from skimage.transform import resize

import cv2

#Data visualization
import seaborn as sns

from matplotlib import pyplot as plt

import glob
import os, os.path

#Per modello NN
from tensorflow.keras import layers
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Softmax
from tensorflow.keras.layers import Dropout

import time
import math

print(tf.__version__)

1.15.2


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
class Stats:

  def __init__( self, img_dim, perc_used, batch_size_used, 
                val_acc, val_loss, total_epochs,
                early_stopping_epochs, total_time, model):

    self.img_dim = img_dim           
    self.perc = str(perc_used)
    self.batch_size = str(batch_size_used)
    
    self.val_acc = str(round(val_acc, 5))
    self.val_loss = str(round(val_loss, 5))

    self.total_epochs = str(total_epochs)
    self.early_stopping_epochs = str(early_stopping_epochs)
    self.training_time = str(round(total_time, 5))
    self.model = model

  def myStats(self):
    print("Dimensione immagini: ", self.img_dim)
    print("Percentuale test set: " + self.perc + "%")
    print("Dimensione batch size: " + self.batch_size)

    print("Val accuracy: " + self.val_acc)
    print("Val loss: " + self.val_loss)

    print("Epoche di addestramento utilizzate: " + self.early_stopping_epochs + "/" + self.total_epochs)
    print("Tempo di addestramento: " + self.training_time + " sec")
    print("Model: " + self.model)

print("Done")

Done


In [4]:
#Extract the class from the file name, if the class is the string before che -
def extract_label(from_string):
  position = from_string.index('-') # gets position of the - in the filename
  substring = from_string[0:position]
  return substring

def extract_obf(from_string):
  start_pos = from_string.index('-')
  end_pos = from_string.index('.')
  substring = from_string[(start_pos + 1):end_pos]
  return substring

def mapping_labels_encoded(label_encoder):
  for index in range(len(list(label_encoder.classes_))):
    print(index, end = "-> ")
    print(list(label_encoder.inverse_transform([index]))) 

class TimeHistory(ks.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(predicted_label,
                                100*np.max(predictions_array),
                                true_label),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')


print("Done")

Done


In [5]:
! git clone https://github.com/PaoloDarienzo/DB_Repo.git

Cloning into 'DB_Repo'...
remote: Enumerating objects: 8293, done.
remote: Total 8293 (delta 0), reused 0 (delta 0), pack-reused 8293
Receiving objects: 100% (8293/8293), 124.99 MiB | 16.76 MiB/s, done.
Resolving deltas: 100% (6795/6795), done.
Checking out files: 100% (14100/14100), done.


In [0]:
path, dirs, files = next(os.walk("/content/DB_Repo"))
file_count = len(files)

In [7]:
%cd /content/DB_Repo
!pwd

/content/DB_Repo
/content/DB_Repo


#INIZIO CODICI NN

In [0]:
def codice_7():

  batch_size = 512

  time_steps = 8
  n_features = 8
  size_ts_blocks = 8

  #Unit in first layer
  num_units1 = 141

  new_dim = 64
  MAX_LEN = 64 #fisso
  channels = 1

  n_epochs = 100

  #Considero il primo 20% della lista di dati come test set
  percentage_required = 20 #%

  #COSTANTI E DICHIARAZIONI

  database_list = list()
  labels_list = list()
  obf_list = list()

  #LETTURA E RESIZE IMMAGINI

  print("START IMAGE INPUT")
  #Aggiungo i valori alle liste leggendo i vari files
  for filename in glob.glob('*.npy'):
    temp_img = np.load(filename)
    temp_img = temp_img.reshape((-1, MAX_LEN)).astype('float32') 
    temp_img = cv2.resize(temp_img, (MAX_LEN, new_dim), interpolation=cv2.INTER_CUBIC)
    database_list.append(temp_img)
    #Salvo la label, ossia la classe
    labels_list.append(extract_label(filename))
    #Salvo la lista di offuscatori di ogni file
    obf_list.append(extract_obf(filename))
  print("END IMAGE INPUT")


  #SHUFFLE

  #Ho i valori e le etichette in due liste (+ obf); 
  #le mescolo mantenendo l'ordine tra valore-label
  temp = list(zip(database_list, labels_list, obf_list))
  np.random.shuffle(temp)
  database_list, labels_list, obf_list = zip(*temp)


  #SUDDIVISIONE DATI

  #Suddivido in training set e test set
  assert len(database_list) == len(labels_list) == len(obf_list)

  index_to_split = math.ceil((len(database_list) * percentage_required) / 100)
  indices = [(0, index_to_split - 1), (index_to_split, len(database_list) - 1)]

  test_list, training_list = [database_list[s:e+1] for s,e in indices]
  labels_test_list, labels_training_list = [labels_list[s:e+1] for s,e in indices]
  obf_test_list, obf_training_list = [obf_list[s:e+1] for s,e in indices]

  #Trasformo i valori in numpy.ndarray
  train_images = np.array(training_list)
  test_images = np.array(test_list)

  train_labels = np.array(labels_training_list)
  test_labels = np.array(labels_test_list)

  train_obf = np.array(obf_training_list)
  test_obf = np.array(obf_test_list)

  label_encoder = LabelEncoder()
  label_encoder.fit(train_labels)
  train_labels_encoded = label_encoder.transform(train_labels)
  test_labels_encoded = label_encoder.transform(test_labels)


  #Normalizzazione valori in range 0-1
  train_images = train_images / 65535.0
  test_images = test_images / 65535.0

  n_img, dim1, dim2 = train_images.shape
  n_img2, dim12, dim22 = test_images.shape

  train_images = train_images.reshape(n_img, time_steps, n_features, -1, size_ts_blocks)

  test_images = test_images.reshape(n_img2, time_steps, n_features, -1, size_ts_blocks)

  _, _, _, val_derivato, _ = train_images.shape 

  #Dichiarazione parametri

  n_classes = len(list(label_encoder.classes_))

  modelLSTM = ks.Sequential()

  #no activation selection

  modelLSTM.add(Bidirectional(ConvLSTM2D(num_units1, (3, 3),
                              padding='same', unit_forget_bias='true', activation='relu'), 
                              input_shape=(time_steps, n_features, val_derivato, size_ts_blocks)))
  modelLSTM.add(Flatten())
  modelLSTM.add(Dense(n_classes, activation='softmax'))

  modelLSTM.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  #Validation_data è usato al termine di ogni epoch;
  #Batch size should be (at most) the same number of hidden cells
  es = ks.callbacks.EarlyStopping(monitor='val_loss', patience=5,
                                  mode='auto', restore_best_weights=True, verbose=0)
  time_callback = TimeHistory()

  hist = modelLSTM.fit(train_images, train_labels_encoded, 
                      batch_size = batch_size,
                      validation_data=(test_images, test_labels_encoded), 
                      epochs=n_epochs, shuffle='true',
                      callbacks=[time_callback, es], verbose=0)

  number_of_epochs_it_ran = len(hist.history['loss'])

  time_per_epoch = time_callback.times
  total_time = sum(time_per_epoch)

  test_accuracy = modelLSTM.evaluate(test_images, test_labels_encoded)

  statistiche_modello = Stats(train_images[0].shape, percentage_required, batch_size, test_accuracy[1], test_accuracy[0], n_epochs, number_of_epochs_it_ran, total_time, "model2_bi")

  return statistiche_modello

In [0]:
def codice_8():

  batch_size = 256

  time_steps = 8
  n_features = 16
  size_ts_blocks = 16

  #Unit in first layer
  num_units1 = 141

  new_dim = 256
  MAX_LEN = 64 #fisso
  channels = 1

  n_epochs = 100

  #Considero il primo 20% della lista di dati come test set
  percentage_required = 20 #%

  #COSTANTI E DICHIARAZIONI

  database_list = list()
  labels_list = list()
  obf_list = list()

  #LETTURA E RESIZE IMMAGINI

  print("START IMAGE INPUT")
  #Aggiungo i valori alle liste leggendo i vari files
  for filename in glob.glob('*.npy'):
    temp_img = np.load(filename)
    temp_img = temp_img.reshape((-1, MAX_LEN)).astype('float32') 
    temp_img = cv2.resize(temp_img, (MAX_LEN, new_dim), interpolation=cv2.INTER_CUBIC)
    database_list.append(temp_img)
    #Salvo la label, ossia la classe
    labels_list.append(extract_label(filename))
    #Salvo la lista di offuscatori di ogni file
    obf_list.append(extract_obf(filename))
  print("END IMAGE INPUT")


  #SHUFFLE

  #Ho i valori e le etichette in due liste (+ obf); 
  #le mescolo mantenendo l'ordine tra valore-label
  temp = list(zip(database_list, labels_list, obf_list))
  np.random.shuffle(temp)
  database_list, labels_list, obf_list = zip(*temp)


  #SUDDIVISIONE DATI

  #Suddivido in training set e test set
  assert len(database_list) == len(labels_list) == len(obf_list)

  index_to_split = math.ceil((len(database_list) * percentage_required) / 100)
  indices = [(0, index_to_split - 1), (index_to_split, len(database_list) - 1)]

  test_list, training_list = [database_list[s:e+1] for s,e in indices]
  labels_test_list, labels_training_list = [labels_list[s:e+1] for s,e in indices]
  obf_test_list, obf_training_list = [obf_list[s:e+1] for s,e in indices]

  #Trasformo i valori in numpy.ndarray
  train_images = np.array(training_list)
  test_images = np.array(test_list)

  train_labels = np.array(labels_training_list)
  test_labels = np.array(labels_test_list)

  train_obf = np.array(obf_training_list)
  test_obf = np.array(obf_test_list)

  label_encoder = LabelEncoder()
  label_encoder.fit(train_labels)
  train_labels_encoded = label_encoder.transform(train_labels)
  test_labels_encoded = label_encoder.transform(test_labels)


  #Normalizzazione valori in range 0-1
  train_images = train_images / 65535.0
  test_images = test_images / 65535.0

  n_img, dim1, dim2 = train_images.shape
  n_img2, dim12, dim22 = test_images.shape

  train_images = train_images.reshape(n_img, time_steps, n_features, -1, size_ts_blocks)

  test_images = test_images.reshape(n_img2, time_steps, n_features, -1, size_ts_blocks)

  _, _, _, val_derivato, _ = train_images.shape 

  #Dichiarazione parametri

  n_classes = len(list(label_encoder.classes_))

  modelLSTM = ks.Sequential()

  #no activation selection

  modelLSTM.add(Bidirectional(ConvLSTM2D(num_units1, (3, 3),
                              padding='same', unit_forget_bias='true', activation='relu'), 
                              input_shape=(time_steps, n_features, val_derivato, size_ts_blocks)))
  modelLSTM.add(Flatten())
  modelLSTM.add(Dense(n_classes, activation='softmax'))

  modelLSTM.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  #Validation_data è usato al termine di ogni epoch;
  #Batch size should be (at most) the same number of hidden cells
  es = ks.callbacks.EarlyStopping(monitor='val_loss', patience=5,
                                  mode='auto', restore_best_weights=True, verbose=0)
  time_callback = TimeHistory()

  hist = modelLSTM.fit(train_images, train_labels_encoded, 
                      batch_size = batch_size,
                      validation_data=(test_images, test_labels_encoded), 
                      epochs=n_epochs, shuffle='true',
                      callbacks=[time_callback, es], verbose=0)

  number_of_epochs_it_ran = len(hist.history['loss'])

  time_per_epoch = time_callback.times
  total_time = sum(time_per_epoch)

  test_accuracy = modelLSTM.evaluate(test_images, test_labels_encoded)

  statistiche_modello = Stats(train_images[0].shape, percentage_required, batch_size, test_accuracy[1], test_accuracy[0], n_epochs, number_of_epochs_it_ran, total_time, "model2_bi")

  return statistiche_modello

In [10]:
lista_res = list()

'''
codice1 lstm64
codice2 lstm256
codice3 bilstm64
codice4 bilstm256
codice5 cnn64
codice6 cnn256
codice7 bicnn64
codice8 bicnn256

'''

for count in range(1, 9):

  if count == 1:
    print("Sono in count: " + str(count))
    
  elif count == 2:
    print("Sono in count: " + str(count))

  elif count == 3:
    print("Sono in count: " + str(count))

  elif count == 4:
    print("Sono in count: " + str(count))

  elif count == 5:
    test_text = input ("Are you there?")
    print("Sono in count: " + str(count))
    
  elif count == 6:
    print("Sono in count: " + str(count))

  elif count == 7:
    print("Sono in count: " + str(count))
    lista_res.append(codice_7())
    #lista_res.append(codice_7())
    #lista_res.append(codice_7())

  elif count == 8:
    print("Sono in count: " + str(count))
    lista_res.append(codice_8())
    #lista_res.append(codice_8())
    #lista_res.append(codice_8())

  else:
    print("ERROR")

Sono in count: 1
Sono in count: 2
Sono in count: 3
Sono in count: 4
Are you there?y
Sono in count: 5
Sono in count: 6
Sono in count: 7
START IMAGE INPUT
END IMAGE INPUT
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2820/2820 [==============================] - 2s 707us/sample - loss: 0.5905 - acc: 0.8968
Sono in count: 8
START IMAGE INPUT
END IMAGE INPUT
2820/2820 [==============================] - 3s 1ms/sample - loss: 0.4858 - acc: 0.9170


In [12]:
for elem in lista_res:
  elem.myStats()

Dimensione immagini:  (8, 8, 8, 8)
Percentuale test set: 20%
Dimensione batch size: 512
Val accuracy: 0.89681
Val loss: 0.59053
Epoche di addestramento utilizzate: 30/100
Tempo di addestramento: 477.62928 sec
Model: model2_bi
Dimensione immagini:  (8, 16, 8, 16)
Percentuale test set: 20%
Dimensione batch size: 256
Val accuracy: 0.91702
Val loss: 0.48583
Epoche di addestramento utilizzate: 16/100
Tempo di addestramento: 522.7539 sec
Model: model2_bi
